This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to overview](kochanek-bartels.ipynb)

# Uniform Kochanek--Bartels Splines

As a starting point,
remember the
[tangent vectors of uniform Catmull--Rom splines](catmull-rom-properties.ipynb#Tangent-Vectors)
-- see also equation 3 of the paper by
<cite data-cite-t="kochanek1984tcb">Kochanek and Bartels (1984)</cite>:

\begin{equation*}
\boldsymbol{\dot{x}}_i = \frac{\boldsymbol{x}_{i+1} - \boldsymbol{x}_{i-1}}{2},
\end{equation*}

which can be re-written as

\begin{equation*}
\boldsymbol{\dot{x}}_i = \frac{
(\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}.
\end{equation*}

## Parameters

Deriving *TCB splines* is all about inserting the parameters
$T$, $C$ and $B$ into this equation.

### Tension

<cite data-cite-t="kochanek1984tcb">Kochanek and Bartels (1984)</cite>
show the usage of $T$ in equation 4:

\begin{equation*}
\boldsymbol{\dot{x}}_i = (1 - T_i) \frac{
(\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}
\end{equation*}

### Continuity

Up to now,
the goal was to have a continuous first derivative at the control points,
i.e. the incoming and outgoing tangent vectors were identical:

\begin{equation*}
\boldsymbol{\dot{x}}_i = \boldsymbol{\dot{x}}_i^{(-)} = \boldsymbol{\dot{x}}_i^{(+)}
\end{equation*}

This also happens to be the requirement for a spline to be $C^1$ continuous.

The *continuity* parameter $C$
allows us to break this continuity if we so desire,
leading to different incoming and outgoing tangent vectors
-- see equations 5 and 6 in the paper by
<cite data-cite-t="kochanek1984tcb">Kochanek and Bartels (1984)</cite>:

\begin{align*}
\boldsymbol{\dot{x}}_i^{(-)} &= \frac{
(1 - C_i) (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(1 + C_i) (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}\\
\boldsymbol{\dot{x}}_i^{(+)} &= \frac{
(1 + C_i) (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(1 - C_i) (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}
\end{align*}

### Bias

<cite data-cite-t="kochanek1984tcb">Kochanek and Bartels (1984)</cite>
show the usage of $B$ in equation 7:

\begin{equation*}
\boldsymbol{\dot{x}}_i = \frac{
(1 + B_i) (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(1 - B_i) (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}
\end{equation*}

### All Three Combined

To get the tangent vectors of a TCB spline,
the three equations can be combined
-- see equations 8 and 9 in the paper by
<cite data-cite="kochanek1984tcb">Kochanek Bartels (1984)</cite>:

\begin{align*}
\boldsymbol{\dot{x}}_i^{(+)} &= \frac{
(1 - T_i) (1 + C_i) (1 + B_i) (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(1 - T_i) (1 - C_i) (1 - B_i) (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}\\
\boldsymbol{\dot{x}}_i^{(-)} &= \frac{
(1 - T_i) (1 - C_i) (1 + B_i) (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(1 - T_i) (1 + C_i) (1 - B_i) (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}
\end{align*}

<div class="alert alert-info">

Note

There is an error in equation (6.11) from 
<cite data-cite-t="millington2009matrices">Millington (2009)</cite>.
All subscripts of $x$ are wrong,
most likely copy-pasted from the preceding equation.

</div>

To simplify the results we will get later, we introduce the following shorthands
<cite data-cite="millington2009matrices">(Millington 2009)</cite>:

\begin{align*}
a_i &= (1 - T_i) (1 + C_i) (1 + B_i),\\
b_i &= (1 - T_i) (1 - C_i) (1 - B_i),\\
c_i &= (1 - T_i) (1 - C_i) (1 + B_i),\\
d_i &= (1 - T_i) (1 + C_i) (1 - B_i),
\end{align*}

which lead to the simplified equations

\begin{align*}
\boldsymbol{\dot{x}}_i^{(+)} &= \frac{
a_i (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
b_i (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}\\
\boldsymbol{\dot{x}}_i^{(-)} &= \frac{
c_i (\boldsymbol{x}_i - \boldsymbol{x}_{i-i}) +
d_i (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}
\end{align*}

## Calculation

The above tangent vectors are sufficient to implement
Kochanek--Bartels splines
via [Hermite splines](hermite.ipynb).
In the rest of this notebook we are deriving
the basis matrix and the basis polynomials
for comparison with other spline types.

In [ ]:
import sympy as sp
sp.init_printing()

As in previous notebooks,
we are using some SymPy helper classes from
[utility.py](utility.py):

In [ ]:
from utility import NamedExpression, NamedMatrix

And again,
we are looking at the fifth spline segment
from $\boldsymbol{x}_4$ to $\boldsymbol{x}_5$
(which can easily be generalized to arbitrary segments).

In [ ]:
x3, x4, x5, x6 = sp.symbols('xbm3:7')

In [ ]:
control_values_KB = sp.Matrix([x3, x4, x5, x6])
control_values_KB

We need three additional parameters per vertex:
$T$, $C$ and $B$.
In our calculation, however,
only the parameters belonging to
$\boldsymbol{x}_4$ and
$\boldsymbol{x}_5$ are relevant:

In [ ]:
T4, T5 = sp.symbols('T4 T5')
C4, C5 = sp.symbols('C4 C5')
B4, B5 = sp.symbols('B4 B5')

Using the shorthands mentioned above ...

In [ ]:
a4 = NamedExpression('a4', (1 - T4) * (1 + C4) * (1 + B4))
b4 = NamedExpression('b4', (1 - T4) * (1 - C4) * (1 - B4))
c5 = NamedExpression('c5', (1 - T5) * (1 - C5) * (1 + B5))
d5 = NamedExpression('d5', (1 - T5) * (1 + C5) * (1 - B5))
display(a4, b4, c5, d5)

... we can define the tangent vectors:

In [ ]:
xd4 = NamedExpression(
    'xdotbm4^(+)',
    sp.S.Half * (a4.name * (x4 - x3) + b4.name * (x5 - x4)))
xd5 = NamedExpression(
    'xdotbm5^(-)',
    sp.S.Half * (c5.name * (x5 - x4)  + d5.name * (x6 - x5)))
display(xd4, xd5)

In [ ]:
display(xd4.subs_symbols(a4, b4))
display(xd5.subs_symbols(c5, d5))

### Basis Matrix

Let's try to find a transformation
from the control values defined above
to *Hermite control values*:

In [ ]:
control_values_H = sp.Matrix([x4, x5, xd4.name, xd5.name])
M_KBtoH = NamedMatrix(r'{M_{\text{KB$,4\to$H}}}', 4, 4)
NamedMatrix(control_values_H, M_KBtoH.name * control_values_KB)

If we substitute the above definitions of
$\boldsymbol{\dot{x}}_4$ and $\boldsymbol{\dot{x}}_5$,
we can obtain the matrix elements:

In [ ]:
M_KBtoH.expr = sp.Matrix([
    [expr.coeff(cv) for cv in control_values_KB]
    for expr in control_values_H.subs([xd4.args, xd5.args]).expand()])
M_KBtoH.pull_out(sp.S.Half)

Once we have a way to get Hermite control values,
we can use the Hermite basis matrix from the
[notebook about uniform cubic Hermite splines](hermite-uniform.ipynb#Basis-Matrix) ...

In [ ]:
M_H = NamedMatrix(
    r'{M_\text{H}}',
    sp.Matrix([[ 2, -2,  1,  1],
               [-3,  3, -2, -1],
               [ 0,  0,  1,  0],
               [ 1,  0,  0,  0]]))
M_H

... to calculate the basis matrix for Kochanek--Bartels splines:

In [ ]:
M_KB = NamedMatrix(r'{M_{\text{KB},4}}', M_H.name * M_KBtoH.name)
M_KB

In [ ]:
M_KB = M_KB.subs_symbols(M_H, M_KBtoH).doit()
M_KB.pull_out(sp.S.Half)

And for completeness' sake, its inverse looks like this:

In [ ]:
M_KB.I

### Basis Polynomials

In [ ]:
t = sp.symbols('t')

Multiplication with the
[monomial basis](polynomials.ipynb)
leads to the basis functions:

In [ ]:
b_KB = NamedMatrix(
    r'{b_{\text{KB},4}}',
    sp.Matrix([t**3, t**2, t, 1]).T * M_KB.expr)
b_KB.T.pull_out(sp.S.Half)

To be able to plot the basis functions,
let's substitute $a_4$, $b_4$, $c_5$ and $d_5$ back in:

In [ ]:
b_KB = b_KB.subs_symbols(a4, b4, c5, d5).simplify()

Let's use a helper function from [helper.py](helper.py):

In [ ]:
from helper import plot_basis

In [ ]:
labels = sp.symbols('xbm_i-1 xbm_i xbm_i+1 xbm_i+2')

To be able to plot the basis functions,
we have to choose some concrete TCB values.

In [ ]:
plot_basis(
    *b_KB.expr.subs({T4: 0, T5: 0, C4: 0, C5: 1, B4: 0, B5: 0}),
    labels=labels)

In [ ]:
plot_basis(
    *b_KB.expr.subs({T4: 0, T5: 0, C4: 0, C5: -0.5, B4: 0, B5: 0}),
    labels=labels)

Setting all TCB values to zero leads to the
[basis polynomials of uniform Catmull--Rom splines](catmull-rom-uniform.ipynb#Basis-Polynomials).